# 🔥 Fine-Tuning de BLOOM-560M para Transformaciones en Guaraní

## Proyecto Final - PLN e IA

**Objetivo:** Entrenar un modelo ligero multilingüe usando **LoRA (Low-Rank Adaptation)** para mejorar el rendimiento en transformaciones morfológicas del guaraní.

### ¿Por qué BLOOM-560M?
- **Multilingüe:** Entrenado en 46 idiomas (incluye lenguas de bajo recurso)
- **Eficiente:** 560M parámetros (funciona en Colab gratuito con GPU T4)
- **Open-source:** Sin restricciones de uso

### ¿Qué es LoRA?
**LoRA (Low-Rank Adaptation)** entrena solo ~1% de los parámetros del modelo mediante matrices de bajo rango, reduciendo:
- ⏱️ **Tiempo:** 10x más rápido que full fine-tuning
- 💾 **Memoria:** Usa 4-bit quantization (modelo cabe en 2GB VRAM)
- 💰 **Costo:** Sin necesidad de GPUs A100 caras

### Configuración del Experimento
- **Dataset:** AmericasNLP 2025 - Guaraní (train: ~800 ejemplos)
- **Épocas:** 5
- **Método:** LoRA + 4-bit quantization
- **Evaluación:** Dev set (100 ejemplos)

---

## 1. Verificar GPU Disponible

In [ ]:
import torch

print("🔍 Verificando hardware...\n")

# Verificar CUDA
if torch.cuda.is_available():
    print(f"✅ GPU disponible: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   CUDA version: {torch.version.cuda}")
else:
    print("⚠️ GPU no disponible. El entrenamiento será LENTO.")
    print("💡 En Colab: Runtime > Change runtime type > GPU (T4)")

## 2. Instalación de Dependencias

In [ ]:
print("📦 Instalando dependencias para fine-tuning con LoRA...\n")

!pip install -q \
  transformers==4.36.0 \
  datasets==2.16.0 \
  peft==0.7.1 \
  bitsandbytes==0.41.3 \
  accelerate==0.25.0 \
  trl==0.7.4 \
  sentencepiece \
  sacrebleu \
  pandas \
  tqdm

print("\n✅ Dependencias instaladas correctamente")
print("\n📚 Bibliotecas clave:")
print("  • transformers: Modelos y tokenizers")
print("  • peft: LoRA y adaptadores eficientes")
print("  • bitsandbytes: Cuantización 4-bit")
print("  • trl: Trainer optimizado para LLMs")

## 3. Importaciones

In [ ]:
import os
import json
import pandas as pd
import requests
from io import StringIO
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel
)
from trl import SFTTrainer
from datasets import Dataset
from sacrebleu.metrics import BLEU
from tqdm import tqdm

print("✅ Importaciones completadas")

## 4. Configuración de Hugging Face Token

**Nota:** Necesitas un token de Hugging Face para descargar BLOOM.
1. Ve a: https://huggingface.co/settings/tokens
2. Crea un token (Read)
3. Guárdalo en Colab Secrets como `HF_TOKEN`

In [ ]:
from google.colab import userdata

try:
    hf_token = userdata.get("HF_TOKEN")
    os.environ["HF_TOKEN"] = hf_token
    print("✅ Hugging Face token configurado")
except Exception as e:
    print("⚠️ No se encontró HF_TOKEN en Secrets")
    print("💡 Configúralo en: Secrets (🔑) > Add new secret > Name: HF_TOKEN")
    hf_token = None

## 5. Descarga del Dataset AmericasNLP

In [ ]:
BASE_URL = "https://raw.githubusercontent.com/AmericasNLP/americasnlp2025/main/ST2_EducationalMaterials/data/"

def load_dataset_from_url(url: str) -> pd.DataFrame:
    """Descarga dataset TSV desde GitHub."""
    r = requests.get(url)
    r.raise_for_status()
    return pd.read_csv(StringIO(r.text), sep="\t")

print("📥 Descargando dataset AmericasNLP...\n")

datasets_urls = {
    "train": f"{BASE_URL}guarani-train.tsv",
    "dev":   f"{BASE_URL}guarani-dev.tsv",
    "test":  f"{BASE_URL}guarani-test.tsv"
}

datasets = {}
for split, url in datasets_urls.items():
    try:
        df = load_dataset_from_url(url)
        datasets[split] = df
        print(f"✅ {split.upper():5} → {len(df):4} ejemplos | Columnas: {list(df.columns)}")
    except Exception as e:
        print(f"❌ Error cargando {split}: {e}")

print(f"\n📊 Total train: {len(datasets['train'])} ejemplos para fine-tuning")
print(f"📊 Total dev:   {len(datasets['dev'])} ejemplos para validación")

### Vista Previa del Dataset

In [ ]:
print("\n🔍 Ejemplos del train set:\n")
display(datasets["train"].head(5))

print("\n📈 Distribución de transformaciones en train:")
print(datasets["train"]["Change"].value_counts())

## 6. Preparación del Dataset para Fine-Tuning

Convertimos cada ejemplo en un formato de instrucción:
```
### Instrucción:
Transforma la siguiente oración en guaraní según la regla indicada.

### Entrada:
Oración: "Ore ndorombyai kuri"
Regla: TYPE:AFF

### Respuesta:
Ore rombyai kuri
```

In [ ]:
# Diccionario de reglas (mismo del notebook principal)
TRANSFORMATION_RULES = {
    "TYPE:AFF": "Convierte una oración negativa en afirmativa removiendo la negación (ndo-...-i)",
    "TYPE:NEG": "Convierte una oración afirmativa en negativa usando ndo- y -i",
    "TENSE:FUT_SIM": "Transforma el verbo al futuro simple agregando -ta",
    "TENSE:PAST": "Convierte la oración al pasado usando kuri",
    "PERSON:1_PL_INC": "Cambia el sujeto a primera persona plural inclusiva (ñande)",
    "PERSON:1_PL_EXC": "Cambia el sujeto a primera persona plural exclusiva (ore)",
    "PERSON:3": "Cambia el sujeto a tercera persona singular (ha'e)"
}

def format_instruction(row: pd.Series) -> str:
    """Convierte una fila del dataset en formato de instrucción."""
    rule_desc = TRANSFORMATION_RULES.get(row["Change"], row["Change"])
    
    return f"""### Instrucción:
Transforma la siguiente oración en guaraní según la regla indicada.

### Entrada:
Oración: "{row["Source"]}"
Regla: {row["Change"]} ({rule_desc})

### Respuesta:
{row["Target"]}"""


print("🔧 Formateando dataset para fine-tuning...\n")

# Formatear train y dev
train_texts = [format_instruction(row) for _, row in datasets["train"].iterrows()]
dev_texts = [format_instruction(row) for _, row in datasets["dev"].iterrows()]

# Crear datasets de Hugging Face
train_dataset = Dataset.from_dict({"text": train_texts})
dev_dataset = Dataset.from_dict({"text": dev_texts})

print(f"✅ Train dataset: {len(train_dataset)} ejemplos")
print(f"✅ Dev dataset:   {len(dev_dataset)} ejemplos")

print("\n📝 Ejemplo formateado:")
print("=" * 70)
print(train_dataset[0]["text"])
print("=" * 70)

## 7. Carga del Modelo Base (BLOOM-560M) con Cuantización 4-bit

In [ ]:
MODEL_NAME = "bigscience/bloom-560m"

print(f"🤖 Cargando modelo: {MODEL_NAME}\n")

# Configuración de cuantización 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Cuantización 4-bit
    bnb_4bit_quant_type="nf4",              # Tipo: NormalFloat4
    bnb_4bit_compute_dtype=torch.float16,   # Computación en FP16
    bnb_4bit_use_double_quant=True          # Doble cuantización
)

# Cargar modelo con cuantización
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    token=hf_token
)

# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    token=hf_token
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("\n✅ Modelo y tokenizer cargados")
print(f"📊 Parámetros totales: {model.num_parameters() / 1e6:.0f}M")
print(f"💾 Tamaño en memoria: ~2GB (cuantizado 4-bit)")

## 8. Configuración de LoRA

**LoRA** añade matrices de bajo rango entrenables a las capas de atención del modelo.

In [ ]:
print("⚙️ Configurando LoRA...\n")

# Preparar modelo para k-bit training
model = prepare_model_for_kbit_training(model)

# Configuración de LoRA
lora_config = LoraConfig(
    r=16,                           # Rango de las matrices LoRA
    lora_alpha=32,                  # Escalado de LoRA
    target_modules=[                # Capas a adaptar
        "query_key_value",          # Atención (específico de BLOOM)
    ],
    lora_dropout=0.05,              # Dropout
    bias="none",                    # No entrenar bias
    task_type="CAUSAL_LM"           # Tipo de tarea
)

# Aplicar LoRA al modelo
model = get_peft_model(model, lora_config)

# Mostrar parámetros entrenables
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print("✅ LoRA configurado correctamente\n")
print(f"📊 Parámetros totales:      {total_params / 1e6:.1f}M")
print(f"🎯 Parámetros entrenables:  {trainable_params / 1e6:.1f}M ({100 * trainable_params / total_params:.2f}%)")
print(f"💡 Eficiencia: Entrenamos solo el {100 * trainable_params / total_params:.2f}% del modelo")

## 9. Configuración del Entrenamiento

In [ ]:
OUTPUT_DIR = "./bloom-560m-guarani-lora"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # Hiperparámetros principales
    num_train_epochs=5,                 # 5 épocas
    per_device_train_batch_size=4,      # Batch size por GPU
    gradient_accumulation_steps=4,      # Simular batch=16
    learning_rate=2e-4,                 # Learning rate
    
    # Optimización
    optim="paged_adamw_8bit",           # Optimizador eficiente
    warmup_steps=50,                    # Warmup
    max_grad_norm=0.3,                  # Gradient clipping
    
    # Logging y guardado
    logging_steps=25,                   # Log cada 25 pasos
    save_strategy="epoch",              # Guardar cada época
    evaluation_strategy="epoch",        # Evaluar cada época
    
    # Eficiencia
    fp16=True,                          # Mixed precision
    group_by_length=True,               # Agrupar por longitud
    
    # Otros
    report_to="none",                   # Sin wandb
    load_best_model_at_end=True,        # Cargar mejor modelo
    metric_for_best_model="loss",       # Métrica
)

print("✅ Training arguments configurados")
print(f"\n📋 Configuración:")
print(f"   Épocas: {training_args.num_train_epochs}")
print(f"   Batch size efectivo: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Pasos totales: ~{len(train_dataset) * training_args.num_train_epochs // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

## 10. Inicialización del Trainer

In [ ]:
print("🎯 Inicializando SFTTrainer...\n")

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    dataset_text_field="text",
    max_seq_length=512,
    packing=False,
)

print("✅ Trainer inicializado y listo para entrenar")

## 11. 🚀 Entrenamiento del Modelo

**Tiempo estimado:** ~30-45 minutos en Colab con GPU T4 (5 épocas, ~800 ejemplos)

⚠️ **Nota:** El entrenamiento puede tardar. Ve por un café ☕

In [ ]:
print("="*70)
print("🔥 INICIANDO FINE-TUNING")
print("="*70)
print(f"Modelo: {MODEL_NAME}")
print(f"Dataset: {len(train_dataset)} ejemplos")
print(f"Épocas: {training_args.num_train_epochs}")
print(f"Método: LoRA (r={lora_config.r}, alpha={lora_config.lora_alpha})")
print("="*70 + "\n")

# ENTRENAR
trainer.train()

print("\n" + "="*70)
print("✅ FINE-TUNING COMPLETADO")
print("="*70)

## 12. Guardar el Modelo Fine-Tuned

In [ ]:
print("💾 Guardando modelo fine-tuned...\n")

# Guardar adaptadores LoRA
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Modelo guardado en: {OUTPUT_DIR}")
print(f"📦 Tamaño: ~50MB (solo adaptadores LoRA)")

# Opcional: Descargar modelo
print("\n📥 Para descargar el modelo, ejecuta:")
print(f"!zip -r {OUTPUT_DIR}.zip {OUTPUT_DIR}")
print("from google.colab import files")
print(f"files.download('{OUTPUT_DIR}.zip')")

## 13. Evaluación del Modelo Fine-Tuned

Comparamos el modelo fine-tuned con el baseline (Claude Few-Shot: 50%)

In [ ]:
print("📊 Evaluando modelo fine-tuned en dev set...\n")

# Crear pipeline de generación
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0.1,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id
)

def extract_response(generated_text: str, input_prompt: str) -> str:
    """Extrae la respuesta del texto generado."""
    # Remover el prompt original
    response = generated_text.replace(input_prompt, "").strip()
    
    # Tomar solo la primera línea después de "### Respuesta:"
    if "### Respuesta:" in response:
        response = response.split("### Respuesta:")[-1].strip()
    
    # Tomar solo hasta el primer salto de línea
    response = response.split("\n")[0].strip()
    
    return response


# Evaluar en dev set (primeros 50 para rapidez)
dev_sample = datasets["dev"].head(50)
results = []

print(f"Evaluando {len(dev_sample)} ejemplos...\n")

for idx, row in tqdm(dev_sample.iterrows(), total=len(dev_sample)):
    # Crear prompt sin respuesta
    rule_desc = TRANSFORMATION_RULES.get(row["Change"], row["Change"])
    input_prompt = f"""### Instrucción:
Transforma la siguiente oración en guaraní según la regla indicada.

### Entrada:
Oración: "{row["Source"]}"
Regla: {row["Change"]} ({rule_desc})

### Respuesta:
"""
    
    # Generar
    outputs = pipe(input_prompt)
    generated = outputs[0]["generated_text"]
    
    # Extraer respuesta
    prediction = extract_response(generated, input_prompt)
    
    # Comparar
    correct = prediction.lower().strip() == row["Target"].lower().strip()
    
    results.append({
        "id": row["ID"],
        "source": row["Source"],
        "change": row["Change"],
        "target": row["Target"],
        "prediction": prediction,
        "correct": correct
    })

# Calcular métricas
correct_count = sum(r["correct"] for r in results)
total = len(results)
accuracy = (correct_count / total) * 100

# BLEU
bleu = BLEU()
predictions = [r["prediction"] for r in results]
references = [[r["target"]] for r in results]
bleu_score = bleu.corpus_score(predictions, references).score

print("\n" + "="*70)
print("RESULTADOS DE EVALUACIÓN")
print("="*70)
print(f"\n🎯 Accuracy: {accuracy:.2f}% ({correct_count}/{total})")
print(f"📊 BLEU Score: {bleu_score:.2f}")

print("\n📈 Comparación con baseline:")
print(f"   Claude 3.5 Few-Shot: 50.0% (mejor anterior)")
print(f"   BLOOM-560M Fine-Tuned: {accuracy:.2f}%")

if accuracy > 50:
    print("\n🏆 ¡El modelo fine-tuned SUPERÓ el baseline!")
elif accuracy > 40:
    print("\n✅ Rendimiento competitivo con el baseline")
else:
    print("\n⚠️ Rendimiento por debajo del baseline (puede mejorar con más épocas)")

## 14. Ejemplos de Predicciones

In [ ]:
print("\n" + "="*70)
print("EJEMPLOS DE PREDICCIONES")
print("="*70)

for i, result in enumerate(results[:10], 1):
    status = "✅" if result["correct"] else "❌"
    print(f"\n{status} Ejemplo {i}:")
    print(f"   Oración:    {result['source']}")
    print(f"   Regla:      {result['change']}")
    print(f"   Esperado:   {result['target']}")
    print(f"   Predicción: {result['prediction']}")

## 15. Guardar Resultados de Evaluación

In [ ]:
import json

# Guardar resultados
eval_results = {
    "model": MODEL_NAME,
    "method": "LoRA Fine-Tuning",
    "epochs": training_args.num_train_epochs,
    "train_size": len(train_dataset),
    "eval_size": len(results),
    "accuracy": accuracy,
    "bleu": bleu_score,
    "correct": correct_count,
    "total": total,
    "predictions": results
}

with open("finetuning_results.json", "w", encoding="utf-8") as f:
    json.dump(eval_results, f, ensure_ascii=False, indent=2)

print("✅ Resultados guardados en: finetuning_results.json")

# Descargar
try:
    from google.colab import files
    files.download("finetuning_results.json")
    print("📥 Archivo descargado")
except:
    print("💡 Descarga manual disponible en archivos")

## 16. Tabla Comparativa Final

Comparamos todas las estrategias probadas en el proyecto.

In [ ]:
import pandas as pd

# Resultados anteriores del notebook principal
comparison_data = [
    {"Modelo": "GPT-3.5 Turbo", "Estrategia": "Zero-Shot", "Accuracy (%)": 0.0, "BLEU": 59.46},
    {"Modelo": "GPT-3.5 Turbo", "Estrategia": "Few-Shot", "Accuracy (%)": 10.0, "BLEU": 0.0},
    {"Modelo": "GPT-3.5 Turbo", "Estrategia": "Semantic RAG", "Accuracy (%)": 0.0, "BLEU": 59.46},
    {"Modelo": "GPT-3.5 Turbo", "Estrategia": "Hybrid RAG", "Accuracy (%)": 10.0, "BLEU": 0.0},
    {"Modelo": "Claude 3.5 Sonnet", "Estrategia": "Zero-Shot", "Accuracy (%)": 30.0, "BLEU": 0.0},
    {"Modelo": "Claude 3.5 Sonnet", "Estrategia": "Few-Shot", "Accuracy (%)": 50.0, "BLEU": 0.0},
    {"Modelo": "Claude 3.5 Sonnet", "Estrategia": "Semantic RAG", "Accuracy (%)": 20.0, "BLEU": 0.0},
    {"Modelo": "Claude 3.5 Sonnet", "Estrategia": "Hybrid RAG", "Accuracy (%)": 40.0, "BLEU": 0.0},
    # Agregar resultado del fine-tuning
    {"Modelo": "BLOOM-560M", "Estrategia": "Fine-Tuning (LoRA)", "Accuracy (%)": round(accuracy, 2), "BLEU": round(bleu_score, 2)}
]

comparison_df = pd.DataFrame(comparison_data)

# Ordenar por accuracy
comparison_df = comparison_df.sort_values("Accuracy (%)", ascending=False)

print("\n" + "="*70)
print("TABLA COMPARATIVA COMPLETA - TODOS LOS EXPERIMENTOS")
print("="*70)
print(comparison_df.to_string(index=False))

# Identificar mejor
best = comparison_df.iloc[0]
print("\n" + "="*70)
print("🏆 MEJOR CONFIGURACIÓN")
print("="*70)
print(f"Modelo:      {best['Modelo']}")
print(f"Estrategia:  {best['Estrategia']}")
print(f"Accuracy:    {best['Accuracy (%)']}%")
print(f"BLEU:        {best['BLEU']}")

## 17. Conclusiones del Fine-Tuning

### ¿Qué aprendimos?

1. **Fine-tuning con LoRA es eficiente:**
   - Entrenar solo ~1% de parámetros
   - Funciona en Colab gratuito (GPU T4)
   - Modelo final: ~50MB (vs 1.1GB del modelo completo)

2. **BLOOM-560M es competitivo para guaraní:**
   - Modelo multilingüe pre-entrenado
   - Se adapta bien con pocos ejemplos (~800)

3. **Comparación con baseline:**
   - Si accuracy > 50%: Fine-tuning superó a Claude Few-Shot
   - Si accuracy < 50%: Necesita más épocas o datos

### Próximos pasos

Para mejorar aún más:
- ✅ Usar todo el train set (hecho)
- ✅ Probar más épocas (5 épocas completadas)
- 🔄 Ajustar hiperparámetros (learning rate, r, alpha)
- 🔄 Probar modelos más grandes (BLOOM-1b7, mGPT)
- 🔄 Data augmentation (generar más ejemplos sintéticos)

---

## 🎉 ¡Experimento completado!

**Archivos generados:**
- `bloom-560m-guarani-lora/` - Modelo fine-tuned
- `finetuning_results.json` - Resultados de evaluación

**Para usar el modelo:**
```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-560m")
model = PeftModel.from_pretrained(base_model, "./bloom-560m-guarani-lora")
tokenizer = AutoTokenizer.from_pretrained("./bloom-560m-guarani-lora")
```